In [1]:
def getAV(coord):
    from astroquery.irsa_dust import IrsaDust
    from astropy.coordinates import Angle,ICRS,SkyCoord
#    print(coord)
#    C = SkyCoord(coord, frame = 'icrs')
#    print(C)
    table = IrsaDust.get_extinction_table(coord,show_progress = False)
#    print(table['A_SandF'][2])
    return (table['A_SandF'][2])


In [2]:
def annulusmedian(distance,ra_central,dec_central):
    from astropy.coordinates import Angle,ICRS,SkyCoord
    import numpy as np
    import math
    from astropy import units as u
    from astropy import coordinates
#    print('starting annulusmedian ', distance, ra_central, dec_central)
#    ra='12h00m00.0s'
#    dec='00d00m00.00s'
#    distance=60.0    
    degreedistance=distance/60.0

    """
        Gets 360 coordinates a specified distance away from the center of the galaxy
        Inputs:
            distance: distance in arcminutes
            ra_central: right ascension component of the coordinate
            dec_central: declination component of the coordinate
        Outputs:
            ring_coords: list of coordinates *distance* arcminutes away from the center of the specified galaxy

    """

    ring_coords = [None]*360 
    directions = [i for i in range(360)]
    AVloop = [i for i in range(360)]

#    print('just before skycoord ',ra_central, ' ', dec_central)

    inputcoord=SkyCoord(ra_central,dec_central,frame='fk5')
#    print(inputcoord)

    for direction in directions:
#        print(direction)
        newdec=inputcoord.dec+Angle(degreedistance*math.cos(direction*2.0*3.14159/360.0),unit=u.degree)
#        print(newdec)
        newra=inputcoord.ra+Angle(math.cos(newdec.value*2.0*3.14159/360.0)*degreedistance*math.sin(direction*2.0*3.14159/360.0),u.degree)
#        print(newra)
         
        ring_coords[direction] = SkyCoord(ra=newra, dec=newdec)
#        print(ring_coords)
#    print(ring_coords)
        AVloop[direction]=getAV(ring_coords[direction])

    AVmedian=np.median(AVloop)
    AVstddev=np.std(AVloop)

    return AVmedian, AVstddev; #performs transformation of initial coordinate into cardinal coordinates


In [3]:
    import math
    import sys
    from astropy import units as u
    from astropy import coordinates
    from astropy.coordinates import Angle,ICRS,SkyCoord
    import numpy as np
    import pandas as pd


In [7]:
    inFile = 'Brown_Walker_table_M106.dat'
    inTable = pd.read_csv(inFile,header=None,delimiter=' ')
    names = inTable.iloc[:,0]
    print(inTable)
    ra = Angle(inTable.iloc[:,1])
    dec = Angle(inTable.iloc[:,2])
    distances = inTable.iloc[:,3]
    cardinalAV = inTable.iloc[:,4]
    cardinalAVspread = inTable.iloc[:,5]
    sourceCoords = SkyCoord(ra,dec,frame='fk5')


      0               1              2   3      4      5                  6
0  M106  12h18m57.5046s  +47d18m14.303  35  0.045  0.005  Brown_Walker_2020
